<a href="https://colab.research.google.com/github/ViniciusCastillo/BootcampAlura_ProjetoModulo5/blob/main/Notebooks/testes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import plot_tree
from sklearn.tree import export_text
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import cross_validate
from sklearn.model_selection import GridSearchCV
from sklearn.feature_selection import SelectFpr
from sklearn.feature_selection import SelectFromModel
from sklearn.pipeline import Pipeline
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.metrics import classification_report

In [3]:
dados = pd.read_excel("https://github.com/ViniciusCastillo/BootcampAlura_ProjetoModulo5/blob/main/dados/Kaggle_Sirio_Libanes_ICU_Prediction.xlsx?raw=true")

In [4]:
print("Tratamento da Base com Label Encoder")

pacientes_UTI = dados[['PATIENT_VISIT_IDENTIFIER','ICU']].query('ICU == 1').groupby('PATIENT_VISIT_IDENTIFIER').min()
dados_tratados = dados.query('ICU != 1').drop('ICU', axis=1)
dados_tratados = dados_tratados.join(pacientes_UTI, on='PATIENT_VISIT_IDENTIFIER', how='left')
dados_tratados['ICU'] = dados_tratados['ICU'].fillna(0) 
print("\nRemovemos as linhas com ICU(UTI) igual a 1 e remarcamos a coluna com base no PATIENT_VISIT_IDENTIFIER que chegou na UTI")
print(f"Distribuição de ICU na base tratada (%)\n{dados_tratados['ICU'].value_counts(normalize=True)*100}")

features_continuas_colunas = dados_tratados.iloc[:, 13:-2].columns
features_continuas = dados_tratados.groupby("PATIENT_VISIT_IDENTIFIER",as_index=False)[features_continuas_colunas].fillna(method='bfill').fillna(method='ffill')
features_categoricas = dados_tratados.iloc[:, :13]
saida = dados_tratados.iloc[:, -2:]
dados_tratados = pd.concat([features_categoricas, features_continuas, saida], ignore_index=True,axis=1)
dados_tratados.columns = dados.columns
print("\nAjustamos os valores continuos que estavam com Nam para o valor anterior ou posterior")

descricao = dados_tratados.describe().T
colunas_sem_variacao = descricao[descricao['min'] == descricao['max']].index
if len(colunas_sem_variacao) !=0:
  dados_tratados.drop(colunas_sem_variacao, axis=1)
  print("\nRemovemos as colunas que os valores são iguais para todas as linhas")

linhas_com_nam = dados_tratados.describe(include='all').loc['count'].max()-dados_tratados.describe(include='all').loc['count'].min()
if linhas_com_nam !=0:
  if linhas_com_nam <= len(dados_tratados)*.1:
    dados_tratados.dropna(inplace=True)
    print(f"\nAs linhas ainda com Nam ({linhas_com_nam} linhas, {linhas_com_nam/len(dados_tratados):.2%} do total) foram eliminadas")
  else:
    print(f"\nTemos linhas ainda com Nam ({linhas_com_nam} linhas, {linhas_com_nam/len(dados_tratados):.2%} do total) precisam ser tratadas")

dados_tratados.reset_index(drop=True, inplace=True)
print(f"\nO index foi resetado: {dados_tratados.index}")

colunas_categoricas = list(set(dados_tratados.columns)-set(dados_tratados.describe().columns)-{'WINDOW'})
if len(colunas_categoricas) ==1:
  LE = preprocessing.LabelEncoder()
  LE.fit(np.ravel(dados_tratados[colunas_categoricas]))
  dados_LE = dados_tratados.copy()
  dados_LE[colunas_categoricas] = LE.transform(np.ravel(dados_LE[colunas_categoricas]))
  print(f"\nColuna com objeto categórico ({colunas_categoricas[0]}) foi transformada em numérica")
  print(f"\nFormato final do DataFrame dados_LE: {dados_LE.shape}")
else:
  print(f"\nColunas com objetos categóricos precisam ser tratados: {', '.join(colunas_categoricas)}")
  
print(f"\nFormato final do DataFrame dados_tratados: {dados_tratados.shape}")

Tratamento da Base com Label Encoder

Removemos as linhas com ICU(UTI) igual a 1 e remarcamos a coluna com base no PATIENT_VISIT_IDENTIFIER que chegou na UTI
Distribuição de ICU na base tratada (%)
0.0    67.375887
1.0    32.624113
Name: ICU, dtype: float64

Ajustamos os valores continuos que estavam com Nam para o valor anterior ou posterior

Removemos as colunas que os valores são iguais para todas as linhas

As linhas ainda com Nam (5.0 linhas, 0.36% do total) foram eliminadas

O index foi resetado: RangeIndex(start=0, stop=1405, step=1)

Coluna com objeto categórico (AGE_PERCENTIL) foi transformada em numérica

Formato final do DataFrame dados_LE: (1405, 231)

Formato final do DataFrame dados_tratados: (1405, 231)


In [5]:
print(f"Tratamento da Base com One Hot Encoder")

if len(colunas_categoricas) ==1:
  categorica = np.array(dados_tratados[colunas_categoricas]).reshape(-1, 1)
  OHE = preprocessing.OneHotEncoder()
  categorica_OHE = pd.DataFrame(OHE.fit_transform(categorica).toarray())
  dados_OHE = pd.concat([dados_tratados.drop(colunas_categoricas, axis=1), categorica_OHE], ignore_index=True, axis=1)
  colunas = list(dados_tratados.columns)
  colunas.remove(colunas_categoricas[0])
  colunas_novas = list(dados_tratados[colunas_categoricas[0]].unique())
  colunas_novas.sort()
  colunas.extend(colunas_novas)
  dados_OHE.columns = colunas
  print(f"\nTrocamos o campo AGE_PERCENTIL pelos campos binários {', '.join(colunas_novas)}")
  print(f"\nFormato final do DataFrame dados_OHE: {dados_OHE.shape}")
else:
  print(f"\nColunas com objetos categóricos precisam ser tratados: {', '.join(colunas_categoricas)}")

Tratamento da Base com One Hot Encoder

Trocamos o campo AGE_PERCENTIL pelos campos binários 10th, 20th, 30th, 40th, 50th, 60th, 70th, 80th, 90th, Above 90th

Formato final do DataFrame dados_OHE: (1405, 240)


In [6]:
class modelos:
  def __init__(self, a, b, **kwargs):
    self.a = a
    self.b = b
  def metodo_a (self, c):
    return self.a*self.b*c

In [7]:
a = modelos(10,2)

In [8]:
a.metodo_a(1)

20

In [43]:
dicionario  = {'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']}
y = dados_tratados_OHE.query('WINDOW == "0-2"')["ICU"]
X = dados_tratados_OHE.query('WINDOW == "0-2"').drop(["ICU","WINDOW","PATIENT_VISIT_IDENTIFIER"], axis=1)
y_OHE = dados_tratados_OHE.query('WINDOW == "0-2"')["ICU"]
X_OHE = dados_tratados_OHE.query('WINDOW == "0-2"').drop(["ICU","WINDOW","PATIENT_VISIT_IDENTIFIER"], axis=1)
cv = RepeatedKFold(n_splits = 5, n_repeats=10)
grid = GridSearchCV(LogisticRegression(max_iter=6000),dicionario, scoring="roc_auc",cv=cv)
melhores = {'base':[],'selecionador':[],'reescala':[],'modelo':[],'auc_roc':[],'desvio_padrao':[]}
for X_, y_, base in [(X, y, ''), (X_OHE, y_OHE, '_OHE')]:
  LR = LogisticRegression(max_iter=6000).fit(X_,y_)
  for i in range(-3,4):
    threshold_i = str(1+i/10)+'*mean'
    seletor = SelectFromModel(LR, threshold=threshold_i).fit(X_,y_)
    X_selecionado = seletor.transform(X_)
    for item in ['nenhuma',preprocessing.StandardScaler(),preprocessing.Normalizer()]:
      if item != 'nenhuma':
        X_revisto = item.fit_transform(X_selecionado)
      else:
        X_revisto = X_selecionado
      resultados= grid.fit(X_revisto,y_)
      desvio = resultados.cv_results_['std_test_score'][resultados.best_index_]
      melhores['base'].append('dados_tratados'+base)
      melhores['selecionador'].append(seletor)
      melhores['reescala'].append(item)
      melhores['modelo'].append(resultados.best_params_)
      melhores['auc_roc'].append(resultados.best_score_)
      melhores['desvio_padrao'].append(desvio)
      print(f'fim {i},{item},{base}')
melhores = pd.DataFrame(melhores)

fim -3,nenhuma,
fim -3,StandardScaler(),
fim -3,Normalizer(),
fim -2,nenhuma,
fim -2,StandardScaler(),
fim -2,Normalizer(),
fim -1,nenhuma,
fim -1,StandardScaler(),
fim -1,Normalizer(),
fim 0,nenhuma,
fim 0,StandardScaler(),
fim 0,Normalizer(),
fim 1,nenhuma,
fim 1,StandardScaler(),
fim 1,Normalizer(),
fim 2,nenhuma,
fim 2,StandardScaler(),
fim 2,Normalizer(),
fim 3,nenhuma,
fim 3,StandardScaler(),
fim 3,Normalizer(),
fim -3,nenhuma,_OHE
fim -3,StandardScaler(),_OHE
fim -3,Normalizer(),_OHE
fim -2,nenhuma,_OHE
fim -2,StandardScaler(),_OHE
fim -2,Normalizer(),_OHE
fim -1,nenhuma,_OHE
fim -1,StandardScaler(),_OHE
fim -1,Normalizer(),_OHE
fim 0,nenhuma,_OHE
fim 0,StandardScaler(),_OHE
fim 0,Normalizer(),_OHE
fim 1,nenhuma,_OHE
fim 1,StandardScaler(),_OHE
fim 1,Normalizer(),_OHE
fim 2,nenhuma,_OHE
fim 2,StandardScaler(),_OHE
fim 2,Normalizer(),_OHE
fim 3,nenhuma,_OHE
fim 3,StandardScaler(),_OHE
fim 3,Normalizer(),_OHE


In [44]:
teste = melhores.sort_values('auc_roc')
teste['inferior']=teste['auc_roc']-2*teste['desvio_padrao']
teste.tail()

,base,selecionador,reescala,modelo,auc_roc,desvio_padrao,inferior
33,dados_tratados_OHE,SelectFromModel(estimator=LogisticRegression(m...,nenhuma,{'solver': 'lbfgs'},0.794386,0.048228,0.697929
39,dados_tratados_OHE,SelectFromModel(estimator=LogisticRegression(m...,nenhuma,{'solver': 'lbfgs'},0.796096,0.044941,0.706214
15,dados_tratados,SelectFromModel(estimator=LogisticRegression(m...,nenhuma,{'solver': 'lbfgs'},0.796490,0.040585,0.715319
36,dados_tratados_OHE,SelectFromModel(estimator=LogisticRegression(m...,nenhuma,{'solver': 'newton-cg'},0.797411,0.056341,0.684728
18,dados_tratados,SelectFromModel(estimator=LogisticRegression(m...,nenhuma,{'solver': 'lbfgs'},0.799609,0.052673,0.694263


In [45]:
teste.sort_values('inferior').tail()

,base,selecionador,reescala,modelo,auc_roc,desvio_padrao,inferior
9,dados_tratados,SelectFromModel(estimator=LogisticRegression(m...,nenhuma,{'solver': 'newton-cg'},0.791242,0.044096,0.703050
12,dados_tratados,SelectFromModel(estimator=LogisticRegression(m...,nenhuma,{'solver': 'newton-cg'},0.792598,0.044625,0.703348
27,dados_tratados_OHE,SelectFromModel(estimator=LogisticRegression(m...,nenhuma,{'solver': 'newton-cg'},0.789682,0.042442,0.704798
39,dados_tratados_OHE,SelectFromModel(estimator=LogisticRegression(m...,nenhuma,{'solver': 'lbfgs'},0.796096,0.044941,0.706214
15,dados_tratados,SelectFromModel(estimator=LogisticRegression(m...,nenhuma,{'solver': 'lbfgs'},0.796490,0.040585,0.715319


In [50]:
teste.loc[15,'selecionador']

SelectFromModel(estimator=LogisticRegression(max_iter=6000),
                threshold='1.2*mean')

In [55]:
dicionario  = {'n_estimators': [10, 50, 100, 150, 200],
               'max_depth': [4, 5, 6, 7, 8],
               'criterion': ['gini', 'entropy'],}
y = dados_tratados_OHE.query('WINDOW == "0-2"')["ICU"]
X = dados_tratados_OHE.query('WINDOW == "0-2"').drop(["ICU","WINDOW","PATIENT_VISIT_IDENTIFIER"], axis=1)
y_OHE = dados_tratados_OHE.query('WINDOW == "0-2"')["ICU"]
X_OHE = dados_tratados_OHE.query('WINDOW == "0-2"').drop(["ICU","WINDOW","PATIENT_VISIT_IDENTIFIER"], axis=1)
cv = RepeatedKFold(n_splits = 5, n_repeats=10)
grid = GridSearchCV(RandomForestClassifier(),dicionario, scoring="roc_auc",cv=cv)
melhores = {'base':[],'selecionador':[],'reescala':[],'modelo':[],'auc_roc':[],'desvio_padrao':[]}
for X_, y_, base in [(X, y, ''), (X_OHE, y_OHE, '_OHE')]:
  RFC = RandomForestClassifier().fit(X_,y_)
  for i in range(-3,4):
    threshold_i = str(1+i/10)+'*mean'
    seletor = SelectFromModel(RFC, threshold=threshold_i).fit(X_,y_)
    X_selecionado = seletor.transform(X_)
    for item in ['nenhuma',preprocessing.StandardScaler(),preprocessing.Normalizer()]:
      if item != 'nenhuma':
        X_revisto = item.fit_transform(X_selecionado)
      else:
        X_revisto = X_selecionado
      resultados= grid.fit(X_revisto,y_)
      desvio = resultados.cv_results_['std_test_score'][resultados.best_index_]
      melhores['base'].append('dados_tratados'+base)
      melhores['selecionador'].append(seletor)
      melhores['reescala'].append(item)
      melhores['modelo'].append(resultados.best_params_)
      melhores['auc_roc'].append(resultados.best_score_)
      melhores['desvio_padrao'].append(desvio)
      print(f'fim {i},{item},{base}')
melhores = pd.DataFrame(melhores)

fim -3,nenhuma,
fim -3,StandardScaler(),
fim -3,Normalizer(),
fim -2,nenhuma,
fim -2,StandardScaler(),
fim -2,Normalizer(),
fim -1,nenhuma,
fim -1,StandardScaler(),
fim -1,Normalizer(),
fim 0,nenhuma,
fim 0,StandardScaler(),
fim 0,Normalizer(),
fim 1,nenhuma,
fim 1,StandardScaler(),
fim 1,Normalizer(),
fim 2,nenhuma,
fim 2,StandardScaler(),
fim 2,Normalizer(),
fim 3,nenhuma,
fim 3,StandardScaler(),
fim 3,Normalizer(),
fim -3,nenhuma,_OHE
fim -3,StandardScaler(),_OHE
fim -3,Normalizer(),_OHE
fim -2,nenhuma,_OHE
fim -2,StandardScaler(),_OHE
fim -2,Normalizer(),_OHE
fim -1,nenhuma,_OHE
fim -1,StandardScaler(),_OHE
fim -1,Normalizer(),_OHE
fim 0,nenhuma,_OHE
fim 0,StandardScaler(),_OHE
fim 0,Normalizer(),_OHE
fim 1,nenhuma,_OHE
fim 1,StandardScaler(),_OHE
fim 1,Normalizer(),_OHE
fim 2,nenhuma,_OHE
fim 2,StandardScaler(),_OHE
fim 2,Normalizer(),_OHE
fim 3,nenhuma,_OHE
fim 3,StandardScaler(),_OHE
fim 3,Normalizer(),_OHE


In [104]:
teste = melhores.sort_values('auc_roc')
teste['inferior']=teste['auc_roc']-2*teste['desvio_padrao']
teste['threshold']=""
for i in teste.index.values:
  teste.loc[i,'threshold'] = teste.loc[i,'selecionador'].threshold
teste['criterion']=""
teste['max_depth']=""
teste['n_estimators']=""
for i in teste.index.values:
  teste.loc[i,'criterion'] = teste.loc[i,'modelo']['criterion']
  teste.loc[i,'max_depth'] = teste.loc[i,'modelo']['max_depth']
  teste.loc[i,'n_estimators'] = teste.loc[i,'modelo']['n_estimators']
teste.drop(['selecionador','modelo'], axis=1, inplace=True)
teste.tail(10)

,base,reescala,auc_roc,desvio_padrao,inferior,threshold,criterion,max_depth,n_estimators
0,dados_tratados,nenhuma,0.796936,0.044331,0.708274,0.7*mean,entropy,8,200
19,dados_tratados,StandardScaler(),0.797676,0.050471,0.696735,1.3*mean,entropy,7,200
4,dados_tratados,StandardScaler(),0.797861,0.052990,0.691881,0.8*mean,entropy,8,150
25,dados_tratados_OHE,StandardScaler(),0.798060,0.053808,0.690444,0.8*mean,entropy,8,150
33,dados_tratados_OHE,nenhuma,0.798272,0.047103,0.704066,1.1*mean,entropy,8,200
36,dados_tratados_OHE,nenhuma,0.798529,0.052524,0.693481,1.2*mean,entropy,7,200
10,dados_tratados,StandardScaler(),0.799837,0.052449,0.694939,1.0*mean,entropy,8,150
27,dados_tratados_OHE,nenhuma,0.800924,0.039990,0.720944,0.9*mean,entropy,8,150
24,dados_tratados_OHE,nenhuma,0.801464,0.050181,0.701101,0.8*mean,entropy,8,200
34,dados_tratados_OHE,StandardScaler(),0.801490,0.047011,0.707469,1.1*mean,entropy,7,200


In [105]:
teste.sort_values('inferior').tail(10)

,base,reescala,auc_roc,desvio_padrao,inferior,threshold,criterion,max_depth,n_estimators
13,dados_tratados,StandardScaler(),0.791038,0.044801,0.701436,1.1*mean,entropy,7,100
41,dados_tratados_OHE,Normalizer(),0.792855,0.045285,0.702285,1.3*mean,entropy,8,200
22,dados_tratados_OHE,StandardScaler(),0.792500,0.044865,0.702770,0.7*mean,entropy,8,200
33,dados_tratados_OHE,nenhuma,0.798272,0.047103,0.704066,1.1*mean,entropy,8,200
34,dados_tratados_OHE,StandardScaler(),0.801490,0.047011,0.707469,1.1*mean,entropy,7,200
0,dados_tratados,nenhuma,0.796936,0.044331,0.708274,0.7*mean,entropy,8,200
9,dados_tratados,nenhuma,0.795187,0.041945,0.711297,1.0*mean,entropy,8,100
12,dados_tratados,nenhuma,0.792396,0.040274,0.711847,1.1*mean,gini,8,100
3,dados_tratados,nenhuma,0.795913,0.041681,0.712550,0.8*mean,entropy,7,200
27,dados_tratados_OHE,nenhuma,0.800924,0.039990,0.720944,0.9*mean,entropy,8,150


In [107]:
teste.sort_values('inferior').head(10)

,base,reescala,auc_roc,desvio_padrao,inferior,threshold,criterion,max_depth,n_estimators
8,dados_tratados,Normalizer(),0.778275,0.058397,0.661482,0.9*mean,entropy,8,150
11,dados_tratados,Normalizer(),0.776804,0.053703,0.669398,1.0*mean,entropy,7,200
38,dados_tratados_OHE,Normalizer(),0.778234,0.053518,0.671198,1.2*mean,entropy,5,200
17,dados_tratados,Normalizer(),0.774842,0.050731,0.673381,1.2*mean,entropy,7,200
32,dados_tratados_OHE,Normalizer(),0.776029,0.050435,0.675158,1.0*mean,entropy,6,150
26,dados_tratados_OHE,Normalizer(),0.773965,0.049372,0.675222,0.8*mean,entropy,8,150
20,dados_tratados,Normalizer(),0.772562,0.048398,0.675765,1.3*mean,entropy,6,100
5,dados_tratados,Normalizer(),0.785253,0.052994,0.679264,0.8*mean,entropy,8,200
29,dados_tratados_OHE,Normalizer(),0.774137,0.046664,0.680809,0.9*mean,entropy,7,150
23,dados_tratados_OHE,Normalizer(),0.778676,0.048783,0.681109,0.7*mean,entropy,8,200


In [74]:
def testa_parametros(modelo, parametros, lista_dados=[(dados_LE, 'LE')], lista_reescalas=['nenhuma'], selecao_menor=-2, selecao_maior=6, passo=2):
  cv = RepeatedStratifiedKFold(n_splits = 5, n_repeats=10)
  grid = GridSearchCV(modelo,parametros, scoring="roc_auc",cv=cv)
  melhores = {'base':[],'selecionador':[],'reescala':[],'modelo':[],'auc_roc':[],'desvio_padrao':[]}
  for dados, base in lista_dados:
    dados = dados.sample(frac=1).reset_index(drop=True)
    y = dados["ICU"]
    X = dados.drop(["ICU","WINDOW","PATIENT_VISIT_IDENTIFIER"], axis=1)
    modelo_ = modelo.fit(X,y)
    for i in range(selecao_menor,selecao_maior+1,passo):
      threshold_i = str(1+i/10)+'*mean'
      seletor = SelectFromModel(modelo_, threshold=threshold_i).fit(X,y)
      X_selecionado = seletor.transform(X)
      for item in lista_reescalas:
        if item != 'nenhuma':
          X_revisto = item.fit_transform(X_selecionado)
        else:
          X_revisto = X_selecionado
        resultados= grid.fit(X_revisto,y)
        desvio = resultados.cv_results_['std_test_score'][resultados.best_index_]
        melhores['base'].append(base)
        melhores['selecionador'].append(seletor)
        melhores['reescala'].append(item)
        melhores['modelo'].append(resultados.best_params_)
        melhores['auc_roc'].append(resultados.best_score_)
        melhores['desvio_padrao'].append(desvio)
        print(f'fim {threshold_i},{item},{base}')
  return pd.DataFrame(melhores)

In [44]:
def classifica_resultados(resultados, ordenar='inferior'): 
  r = resultados.copy()
  r['inferior']=r['auc_roc']-2*r['desvio_padrao']
  r['superior']=r['auc_roc']+2*r['desvio_padrao']
  r['threshold']=""
  for i in r.index.values:
    r.loc[i,'threshold'] = r.loc[i,'selecionador'].threshold
  for titulo in r.loc[0,'modelo'].keys():
    r[titulo]=''
    for i in r.index.values:
      r.loc[i,titulo] = r.loc[i,'modelo'][titulo]
  r.drop(['selecionador','modelo'], axis=1, inplace=True)
  return r.sort_values(ordenar)

In [140]:
dicionario  = {'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']}
r = testa_parametros(LogisticRegression(max_iter=6000), dicionario, lista_dados=[(dados_LE,'LE'),(dados_OHE,'OHE')], 
                     lista_reescalas=['nenhuma',preprocessing.StandardScaler()], selecao_maior=8)

fim -2,nenhuma,LE
fim -2,StandardScaler(),LE
fim 0,nenhuma,LE
fim 0,StandardScaler(),LE
fim 2,nenhuma,LE
fim 2,StandardScaler(),LE
fim 4,nenhuma,LE
fim 4,StandardScaler(),LE
fim 6,nenhuma,LE
fim 6,StandardScaler(),LE
fim 8,nenhuma,LE
fim 8,StandardScaler(),LE
fim -2,nenhuma,OHE
fim -2,StandardScaler(),OHE
fim 0,nenhuma,OHE
fim 0,StandardScaler(),OHE
fim 2,nenhuma,OHE
fim 2,StandardScaler(),OHE
fim 4,nenhuma,OHE
fim 4,StandardScaler(),OHE
fim 6,nenhuma,OHE
fim 6,StandardScaler(),OHE
fim 8,nenhuma,OHE
fim 8,StandardScaler(),OHE


In [142]:
teste = classifica_resultados(r)
teste.tail()

,base,reescala,auc_roc,desvio_padrao,inferior,superior,threshold,solver
20,OHE,nenhuma,0.796693,0.047505,0.701683,0.891704,1.6*mean,newton-cg
14,OHE,nenhuma,0.789033,0.038992,0.711048,0.867017,1.0*mean,newton-cg
18,OHE,nenhuma,0.799670,0.043498,0.712673,0.886666,1.4*mean,sag
6,LE,nenhuma,0.799564,0.042649,0.714267,0.884861,1.4*mean,newton-cg
22,OHE,nenhuma,0.798064,0.037815,0.722434,0.873694,1.8*mean,sag


In [131]:
teste = classifica_resultados(melhores)
teste.tail()

,base,reescala,auc_roc,desvio_padrao,inferior,superior,threshold,criterion,max_depth,n_estimators
0,dados_tratados,nenhuma,0.796936,0.044331,0.708274,0.885598,0.7*mean,entropy,8,200
9,dados_tratados,nenhuma,0.795187,0.041945,0.711297,0.879077,1.0*mean,entropy,8,100
12,dados_tratados,nenhuma,0.792396,0.040274,0.711847,0.872944,1.1*mean,gini,8,100
3,dados_tratados,nenhuma,0.795913,0.041681,0.712550,0.879276,0.8*mean,entropy,7,200
27,dados_tratados_OHE,nenhuma,0.800924,0.039990,0.720944,0.880905,0.9*mean,entropy,8,150


In [8]:
y = dados_OHE.query('WINDOW == "0-2"')["ICU"]
X = dados_OHE.query('WINDOW == "0-2"').drop(["ICU","WINDOW","PATIENT_VISIT_IDENTIFIER"], axis=1)
print(X.shape)
X = SelectFpr().fit_transform(X,y)
X.shape

(352, 237)


/usr/local/lib/python3.7/dist-packages/sklearn/feature_selection/_univariate_selection.py:112: UserWarning: Features [ 15  20  25  30  35  40  45  50  55  60  65  70  75  80  85  90  95 100
 105 110 115 120 125 130 135 140 145 150 155 160 165 170 175 180 185 190] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/feature_selection/_univariate_selection.py:113: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw


(352, 83)

In [17]:
dicionario  = {'n_estimators': [10, 100, 200],
               'max_depth': [4, 6, 8],
               'criterion': ['gini', 'entropy'],}
cv = RepeatedStratifiedKFold(n_splits = 5, n_repeats=10)
grid = GridSearchCV(RandomForestClassifier(),dicionario, scoring="roc_auc",cv=cv)
result = grid.fit(X,y)

In [20]:
result.cv_results_['mean_test_score']

array([0.75160995, 0.78291448, 0.78391469, 0.75764905, 0.79019792,
       0.79111477, 0.7582809 , 0.79377306, 0.79696909, 0.76555307,
       0.77990499, 0.78468288, 0.77386059, 0.78867972, 0.79044968,
       0.7600535 , 0.79241485, 0.79448807])

In [27]:
result.cv_results_['params'][2]

{'criterion': 'gini', 'max_depth': 4, 'n_estimators': 200}

In [36]:
dados_OHE_1 = dados_OHE.query("WINDOW == '0-2'")
dados_OHE_1.set_index('PATIENT_VISIT_IDENTIFIER', inplace=True)
print(dados_OHE_1.shape)
dados_OHE_2 = dados_OHE.query("WINDOW == '2-4'")
dados_OHE_2.set_index('PATIENT_VISIT_IDENTIFIER', inplace=True)
dados_OHE_2.shape

(352, 239)


(325, 239)

In [169]:
def adiciona_janela(dados_anterior, dados_janela, colunas=features_continuas_colunas):
  d1 = dados_janela.set_index('PATIENT_VISIT_IDENTIFIER')
  d2 = dados_anterior.set_index('PATIENT_VISIT_IDENTIFIER')
  sufixo = ' ' + str(dados_anterior.loc[0,"WINDOW"])
  dados_novo = d1.join(d2[colunas], how='left', rsuffix=sufixo).reset_index()

  for _ in colunas:
    coluna = 'var ' + _ + sufixo
    coluna_ = _ + sufixo
    dados_novo[coluna] = dados_novo[_] - dados_novo[coluna_]

  descricao = dados_novo.describe().T
  colunas_sem_variacao = descricao[descricao['min'] == descricao['max']].index
  if len(colunas_sem_variacao) !=0:
    dados_novo.drop(colunas_sem_variacao, axis=1)
    print("\nRemovemos as colunas que os valores são iguais para todas as linhas")

  linhas_com_nam = dados_novo.describe(include='all').loc['count'].max()-dados_novo.describe(include='all').loc['count'].min()
  if linhas_com_nam !=0:
    if linhas_com_nam <= len(dados_novo)*.1:
      dados_novo.dropna(inplace=True)
      print(f"\nAs linhas ainda com Nam ({linhas_com_nam} linhas, {linhas_com_nam/len(dados_novo):.2%} do total) foram eliminadas")
    else:
      print(f"\nTemos linhas ainda com Nam ({linhas_com_nam} linhas, {linhas_com_nam/len(dados_novo):.2%} do total) precisam ser tratadas")
  print(f'\nBase nova de tamanho: {dados_novo.shape}')
  return dados_novo

In [143]:
y = dados_OHE_novo["ICU"]
X = dados_OHE_novo.drop(["ICU","WINDOW","PATIENT_VISIT_IDENTIFIER"], axis=1)
X = SelectFpr().fit_transform(X,y)
X.shape

/usr/local/lib/python3.7/dist-packages/sklearn/feature_selection/_univariate_selection.py:112: UserWarning: Features [ 15  20  25  30  35  40  45  50  55  60  65  70  75  80  85  90  95 100
 105 110 115 120 125 130 135 140 145 150 155 160 165 170 175 180 185 190
 241 246 251 256 261 266 271 276 281 286 291 296 301 306 311 316 321 326
 331 336 341 346 351 356 361 366 371 376 381 386 391 396 401 406 411 416] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/feature_selection/_univariate_selection.py:113: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw


(325, 155)

In [69]:
dicionario  = {'n_estimators': [200],
               'max_depth': [5, 10, None],
               'criterion': ['gini', 'entropy'],}
cv = RepeatedStratifiedKFold(n_splits = 5, n_repeats=10)
grid = GridSearchCV(RandomForestClassifier(),dicionario, scoring="roc_auc",cv=cv)
result = grid.fit(X,y)

In [70]:
result.cv_results_['mean_test_score']

array([0.82392268, 0.82999383, 0.82953753, 0.82324814, 0.82778959,
       0.82940922])

In [71]:
result.best_params_

{'criterion': 'gini', 'max_depth': 10, 'n_estimators': 200}

In [162]:
dicionario  = {'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']}
s = testa_parametros(LogisticRegression(max_iter=6000), dicionario, lista_dados=[(dados_OHE_novo,'OHE')], 
                     lista_reescalas=['nenhuma',preprocessing.StandardScaler()], selecao_maior=8, selecao_menor=2, passo=3)

fim 1.2*mean,nenhuma,OHE
fim 1.2*mean,StandardScaler(),OHE
fim 1.5*mean,nenhuma,OHE
fim 1.5*mean,StandardScaler(),OHE
fim 1.8*mean,nenhuma,OHE
fim 1.8*mean,StandardScaler(),OHE


In [163]:
teste = classifica_resultados(s)
teste

,base,reescala,auc_roc,desvio_padrao,inferior,superior,threshold,solver
1,OHE,StandardScaler(),0.800654,0.050503,0.699648,0.901660,1.2*mean,saga
5,OHE,StandardScaler(),0.816887,0.056648,0.703591,0.930184,1.8*mean,liblinear
3,OHE,StandardScaler(),0.809280,0.048684,0.711911,0.906649,1.5*mean,newton-cg
0,OHE,nenhuma,0.825691,0.044181,0.737329,0.914053,1.2*mean,newton-cg
2,OHE,nenhuma,0.834828,0.045772,0.743284,0.926371,1.5*mean,newton-cg
4,OHE,nenhuma,0.846953,0.042477,0.761999,0.931907,1.8*mean,lbfgs


In [166]:
s.loc[4,'selecionador'].get_feature_names_out()

array(['AGE_ABOVE65', 'GENDER', 'DISEASE GROUPING 1',
       'DISEASE GROUPING 2', 'DISEASE GROUPING 4', 'HTN', 'OTHER',
       'BIC_VENOUS_MEDIAN', 'BIC_VENOUS_MEAN', 'BIC_VENOUS_MIN',
       'BIC_VENOUS_MAX', 'CALCIUM_MEDIAN', 'CALCIUM_MEAN', 'CALCIUM_MIN',
       'CALCIUM_MAX', 'CREATININ_MEDIAN', 'CREATININ_MEAN',
       'CREATININ_MIN', 'CREATININ_MAX', 'GLUCOSE_MEDIAN', 'GLUCOSE_MEAN',
       'GLUCOSE_MIN', 'GLUCOSE_MAX', 'LINFOCITOS_MEDIAN',
       'LINFOCITOS_MEAN', 'LINFOCITOS_MIN', 'LINFOCITOS_MAX',
       'PCR_MEDIAN', 'PCR_MEAN', 'PCR_MIN', 'PCR_MAX', 'POTASSIUM_MEDIAN',
       'POTASSIUM_MEAN', 'POTASSIUM_MIN', 'POTASSIUM_MAX',
       'SODIUM_MEDIAN', 'SODIUM_MEAN', 'SODIUM_MIN', 'SODIUM_MAX',
       'BLOODPRESSURE_DIASTOLIC_MEAN', 'BLOODPRESSURE_SISTOLIC_MEAN',
       'BLOODPRESSURE_DIASTOLIC_MEDIAN', 'BLOODPRESSURE_DIASTOLIC_MIN',
       'BLOODPRESSURE_SISTOLIC_MIN', 'HEART_RATE_MIN', 'TEMPERATURE_MIN',
       'BLOODPRESSURE_SISTOLIC_MAX', 'HEART_RATE_MAX',
       'BLOOD

In [154]:
from sklearn.base import BaseEstimator, TransformerMixin
class RemoveCorrVar(BaseEstimator, TransformerMixin):
    def __init__( self, valor_corte = 0.95):
        self.valor_corte = valor_corte
        
    def fit( self, X, y = None ):
        matrix_corr = X.iloc[:,4:].corr().abs()
        matrix_upper = matrix_corr.where(np.triu(np.ones(matrix_corr.shape), k=1).astype(np.bool))
        self.excluir = [coluna for coluna in matrix_upper.columns if any(matrix_upper[coluna] > self.valor_corte)]
        return self 
    
    def transform(self, X, y = None):
        X = X.drop(self.excluir, axis=1)
        return X

(325, 159)

In [158]:
aa = RemoveCorrVar(.85).fit(dados_OHE_novo.drop(["ICU","WINDOW","PATIENT_VISIT_IDENTIFIER"], axis=1))
X_ = aa.transform(dados_OHE_novo.drop(["ICU","WINDOW","PATIENT_VISIT_IDENTIFIER"], axis=1))
print(X_.shape)
colunas = ['ICU']
colunas.extend(X_.columns)
corr = dados_OHE_novo[colunas].corr().abs()
colunas_ = list(corr.query('ICU != 1 and ICU >= 0.15')['ICU'].index)
len(colunas_)

(325, 239)


30

In [159]:
result = grid.fit(X_[colunas_],y)
result.cv_results_['mean_test_score']

array([0.8323936 , 0.8323936 , 0.82392893, 0.83235442, 0.83215987])

In [160]:
colunas_

['AGE_ABOVE65',
 'HTN',
 'BIC_VENOUS_MEDIAN',
 'CALCIUM_MEDIAN',
 'CREATININ_MEDIAN',
 'GLUCOSE_MEDIAN',
 'LINFOCITOS_MEDIAN',
 'PC02_VENOUS_MEDIAN',
 'PCR_MEDIAN',
 'PLATELETS_MEDIAN',
 'SODIUM_MEDIAN',
 'UREA_MEDIAN',
 'OXYGEN_SATURATION_MIN',
 'BLOODPRESSURE_DIASTOLIC_DIFF',
 'HEART_RATE_DIFF',
 'RESPIRATORY_RATE_DIFF',
 'TEMPERATURE_DIFF',
 'OXYGEN_SATURATION_DIFF',
 '10th',
 '90th',
 'Above 90th',
 'LACTATE_MEDIAN 0-2',
 'LINFOCITOS_MEDIAN 0-2',
 'PC02_VENOUS_MEDIAN 0-2',
 'PCR_MEDIAN 0-2',
 'BLOODPRESSURE_DIASTOLIC_MEAN 0-2',
 'OXYGEN_SATURATION_MEAN 0-2',
 'var LINFOCITOS_MEDIAN',
 'var PCR_MEDIAN',
 'var PLATELETS_MEDIAN']

In [127]:
aaa = dados_OHE_novo['AGE_ABOVE65 0-2'] - dados_OHE_novo['AGE_ABOVE65']
aaa.sum()

0